In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_data=pd.read_csv("train.csv")
test_data=pd.read_csv("test.csv")

In [3]:
train_data.head()

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,2020-01-26,0.0,0.0


In [4]:
test_data.head()

,ForecastId,Province_State,Country_Region,Date
0,1,NaN,Afghanistan,2020-04-02
1,2,NaN,Afghanistan,2020-04-03
2,3,NaN,Afghanistan,2020-04-04
3,4,NaN,Afghanistan,2020-04-05
4,5,NaN,Afghanistan,2020-04-06


In [5]:
train_data.describe()

,Id,ConfirmedCases,Fatalities
count,25040.000000,25040.000000,25040.000000
mean,17824.500000,945.632029,46.689816
std,10300.685154,7205.968068,559.239035
min,1.000000,0.000000,0.000000
25%,8912.750000,0.000000,0.000000
50%,17824.500000,1.000000,0.000000
75%,26736.250000,85.000000,1.000000
max,35648.000000,172348.000000,18849.000000


In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25040 entries, 0 to 25039
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              25040 non-null  int64  
 1   Province_State  10640 non-null  object 
 2   Country_Region  25040 non-null  object 
 3   Date            25040 non-null  object 
 4   ConfirmedCases  25040 non-null  float64
 5   Fatalities      25040 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 1.1+ MB


In [7]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13459 entries, 0 to 13458
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ForecastId      13459 non-null  int64 
 1   Province_State  5719 non-null   object
 2   Country_Region  13459 non-null  object
 3   Date            13459 non-null  object
dtypes: int64(1), object(3)
memory usage: 420.7+ KB


In [8]:
train_data=train_data.drop(["Province_State"],axis=1)
test_data=test_data.drop(["Province_State"],axis=1)

In [9]:
from sklearn.preprocessing import LabelBinarizer
lb=LabelBinarizer()
train_data["Country_Region"]=lb.fit_transform(train_data["Country_Region"])

In [10]:
test_data["Country_Region"]=lb.transform(test_data["Country_Region"])

In [11]:
train_data.head()

,Id,Country_Region,Date,ConfirmedCases,Fatalities
0,1,1,2020-01-22,0.0,0.0
1,2,1,2020-01-23,0.0,0.0
2,3,1,2020-01-24,0.0,0.0
3,4,1,2020-01-25,0.0,0.0
4,5,1,2020-01-26,0.0,0.0


In [12]:
train_data=train_data.drop(["Id"],axis=1)

In [13]:
test_data=test_data.drop(["ForecastId"],axis=1)

In [14]:
train_data.Date=train_data.Date.apply(lambda x:x.split("-"))

In [15]:
train_data.head()

,Country_Region,Date,ConfirmedCases,Fatalities
0,1,"[2020, 01, 22]",0.0,0.0
1,1,"[2020, 01, 23]",0.0,0.0
2,1,"[2020, 01, 24]",0.0,0.0
3,1,"[2020, 01, 25]",0.0,0.0
4,1,"[2020, 01, 26]",0.0,0.0


In [16]:
test_data.Date=test_data.Date.apply(lambda x:x.split("-"))

In [17]:
test_data.head()

,Country_Region,Date
0,1,"[2020, 04, 02]"
1,1,"[2020, 04, 03]"
2,1,"[2020, 04, 04]"
3,1,"[2020, 04, 05]"
4,1,"[2020, 04, 06]"


In [18]:
def month_day(dataset):
    month=[]
    day=[]
    for i in dataset.Date:
        month.append(int(i[1]))
        day.append(int(i[2]))
    dataset["month"]=month
    dataset["day"]=day
    dataset=dataset.drop(["Date"],axis=1)
    return dataset
    


In [19]:
train_data=month_day(train_data)
test_data=month_day(test_data)

In [20]:
train_data["ConfirmedCases"]=train_data["ConfirmedCases"].apply(int)
train_data["Fatalities"]=train_data["Fatalities"].apply(int)

In [21]:
x=train_data.copy()
x=x.drop(["ConfirmedCases","Fatalities"],axis=1)
x.head()

,Country_Region,month,day
0,1,1,22
1,1,1,23
2,1,1,24
3,1,1,25
4,1,1,26


In [22]:
y_cc=train_data["ConfirmedCases"].values
y_ft=train_data["Fatalities"].values

In [23]:
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler()
x=sc.fit_transform(x)

In [24]:
test_data=sc.transform(test_data)

In [25]:
from xgboost import XGBRegressor
reg_cc=XGBRegressor(n_estimators=1500,max_depth=15,random_state=0)

In [26]:
reg_cc.fit(x,y_cc)

[21:54:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=15, min_child_weight=1, missing=None, n_estimators=1500,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [27]:
reg_ff=XGBRegressor(n_estimators=1500,max_depth=15,random_state=0)
reg_ff.fit(x,y_ft)

[21:54:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=15, min_child_weight=1, missing=None, n_estimators=1500,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [28]:
pred_cc=reg_cc.predict(test_data)

In [29]:
pred_cc=np.around(pred_cc)

In [30]:
pred_ff=reg_ff.predict(test_data)

In [31]:
pred_ff=np.around(pred_ff)

In [32]:
pred_cc

array([ 273.,  281.,  299., ..., 5420., 5420., 5420.], dtype=float32)

In [33]:
pred_ff

array([  6.,   6.,   7., ..., 329., 329., 329.], dtype=float32)

In [34]:
submission_data=pd.read_csv("submission.csv")

In [35]:
submission_data.head()

,ForecastId,ConfirmedCases,Fatalities
0,1,1,1
1,2,1,1
2,3,1,1
3,4,1,1
4,5,1,1


In [36]:
submission_data["ConfirmedCases"]=pred_cc
submission_data["Fatalities"]=pred_ff

In [37]:
submission_data.head()

,ForecastId,ConfirmedCases,Fatalities
0,1,273.0,6.0
1,2,281.0,6.0
2,3,299.0,7.0
3,4,349.0,7.0
4,5,367.0,11.0


In [39]:
submission_data.to_csv("submission_data.csv",index=False)